In [26]:
import pandas as pd
import json
data = pd.read_csv("./StatQA/Data/Integrated Dataset/Dataset with Prompt/Test Set/mini-StatQA for zero-shot.csv")


In [27]:
data_json_str = data.to_json(orient='records', lines=True)
data_json = [json.loads(line) for line in data_json_str.splitlines()]


In [34]:
import srsly
data_json[0]
json_data = json.dumps(data_json)
df = pd.read_json(json_data)

/tmp/ipykernel_2843921/3760094643.py:4: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_json(json_data)


In [35]:
df.head()

,dataset,refined_question,relevant_column,task,difficulty,results,prompt
0,Dataset for Admission in the University,Is the variability in GRE scores not significa...,"[{""column_header"": ""GRE Score"", ""is_strata"": f...",Variance Test,hard,"[{""method"": ""Mood Variance Test"", ""result"": ""{...",### Task Description: You need to select relev...
1,Crop Production Dataset,Are the variations in yearly crop yields in th...,"[{""column_header"": ""United States"", ""is_strata...",Variance Test,easy,"[{""method"": ""Mood Variance Test"", ""result"": ""{...",### Task Description: You need to select relev...
2,Terrorist Attacks Dataset,What is the relationship between the Terrorist...,"[{""column_header"": ""Terrorist Death Type : Kil...",Correlation Analysis,easy,"[{""method"": ""Partial Correlation Coefficient"",...",### Task Description: You need to select relev...
3,Customer Personality Analysis Dataset,How skewed is the distribution of spending on ...,"[{""column_header"": ""MntMeatProducts"", ""is_stra...",Descriptive Statistics,easy,"[{""method"": ""Kurtosis"", ""result"": ""5.51672"", ""...",### Task Description: You need to select relev...
4,Income and Happiness Score Dataset,Do observable differences exist in the distrib...,"[{""column_header"": ""happyScore"", ""is_strata"": ...",Distribution Compliance Test,easy,"[{""method"": ""Kolmogorov-Smirnov Test"", ""result...",### Task Description: You need to select relev...


In [21]:
import asyncio
import aiohttp
import srsly
import http.client
import json
from tqdm import tqdm

API_KEY = 'sk-'

BASE_URL = "https://api.gpt.ge/v1/"
# BASE_URL = "https://run.v36.cm/v1"

headers = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json",
}
async def create_completion(session, content):
    try:
        async with session.post(
            url=f"{BASE_URL}chat/completions",
            json={
                # "model": "gpt-4o",
                "model": "deepseek-v3",
                "max_tokens": 30,
                "temperature": 0.00000000000000000000001,
                "messages": [{"role": "user", "content": content['prompt']}],
            },
            headers=headers
        ) as response:
            if response.status == 200:
                result = await response.json()
                completion = result['choices'][0]['message']['content']
                content[0]['model_answer'] = completion
                return completion
            else:
                print(f"请求失败，状态码: {response.status}")
                print(f"响应内容: {await response.text()}")
                content[0]['model_answer'] = ''
                return None
    except Exception as e:
        print(f"请求发生异常: {e}")
        return None
    
async def main():
    max_limits = 250  # 设置一个合理的并发请求数
    semaphore = asyncio.Semaphore(max_limits)
    results = []

    async with aiohttp.ClientSession() as session:
        tasks = []

        for content in data_json:
            task = asyncio.create_task(limited_create_completion(session, content, semaphore))
            tasks.append(task)

        for future in tqdm(asyncio.as_completed(tasks), total=len(tasks)):
            result = await future
            results.append(result)

    return results

async def limited_create_completion(session, content, semaphore):
    async with semaphore:
        return await create_completion(session, content)
    
responses = asyncio.run(main())

RuntimeError: asyncio.run() cannot be called from a running event loop